In [160]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
from transformer.transformer import DataLoader    
from utils.parse_data import load_trained_model
project_root = os.path.abspath("..")

In [183]:
#1.  Load the model that was trained in run_2
RUN = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model, model_info, cfg = load_trained_model(
    run        = RUN,
    model_name = "model_seen9M",
    device     = device,
)
print(f"Loaded model {model_info['model_name']} on {device}  "
      f"(embedding={cfg.n_embd}, block_size={cfg.block_size})")

Loaded model model_seen9M on cpu  (embedding=4, block_size=6)


/Users/corwin/Building/Learning/Transformers_for_Modeling_Decision_Sequences/utils/parse_data.py:426: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(tor

In [184]:
# 2. Build a dataloader with the same B and T used during training
B, T = 300, 6 
loader = DataLoader(
    B             = B,
    T             = T,
    process_rank  = 0,
    num_processes = 1,
    run_number    = RUN,
    suffix        = 'tr',
)


read in 100000 tokens from /Users/corwin/Building/Learning/Transformers_for_Modeling_Decision_Sequences/experiments/run_2/seqs/behavior_run_2tr.txt


In [185]:
# 3. Collect residuals, log-odds, and sequences using passive estimator
from synthetic_data_generation.agent import RFLR_mouse

N_BATCHES = 10  # collect 3000 sequences
all_residuals = []
all_passive_logodds = []  # Ground truth log odds from passive estimator
all_sequences = []  # store the actual RrLlR sequences

# Create an agent instance for passive estimation
# You may need to adjust these parameters based on your training setup
agent = RFLR_mouse(alpha=0.78, beta=2.05, tau=1.43)

model.eval()
with torch.no_grad():
    for batch_idx in range(N_BATCHES):
        x, _ = loader.next_batch()
        x = x.to(device)

        logits, loss, residual = model(x, return_residual=True)
        
        # Reshape residual from [B, T, n_embd] to [n_embd, T] for each batch
        # residual shape: [B, T, n_embd] -> [B, n_embd, T] -> Bx4x6 in our case
        residual_reshaped = residual.permute(0, 2, 1)

        for b in range(residual_reshaped.size(0)):
            all_residuals.append(residual_reshaped[b])            # [4,6]
            
            # Convert tensor indices back to R/L characters and choices/rewards
            vocab = ['R', 'r', 'L', 'l']
            itos = {i: ch for i, ch in enumerate(vocab)}
            sequence_chars = tuple(itos[int(idx)] for idx in x[b])
            all_sequences.append(sequence_chars)               # [6] - actual sequence as chars
            
            # Convert sequence to choices and rewards for passive estimator
            choices = []
            rewards = []
            for char in sequence_chars:
                if char in ['R', 'r']:
                    choices.append(1)  # Right choice
                    rewards.append(1 if char == 'R' else 0)  # Reward if uppercase
                else:  # char in ['L', 'l']
                    choices.append(0)  # Left choice  
                    rewards.append(1 if char == 'L' else 0)  # Reward if uppercase
            
            agent = RFLR_mouse(alpha=0.78, beta=2.05, tau=1.43)
            
            for c, r in zip(choices, rewards):
                agent.update_phi(c, r)
            
            logodds_next = agent.compute_log_odds(choices[-1])
            all_passive_logodds.append(logodds_next)

# ------------------------------------------------------------------
# 4.  Verify every snapshot is n_embd×6 format
# ------------------------------------------------------------------
expected_seq_len = 6
ok = all(residual.shape[1] == expected_seq_len for residual in all_residuals)
print(f"\nAll residual snapshots have sequence length {expected_seq_len}:", ok)
print(f"Total residuals collected: {len(all_residuals)}")
print(f"Each residual shape: {all_residuals[0].shape if all_residuals else 'None'}")
assert ok, f"Found a residual whose sequence length ≠ {expected_seq_len}"



All residual snapshots have sequence length 6: True
Total residuals collected: 3000
Each residual shape: torch.Size([4, 6])


In [186]:
# 5. Deduplicate sequences and show summary
unique = {}
for i, seq in enumerate(all_sequences):
    if seq not in unique:
        unique[seq] = i  # store first occurrence index

unique_indices = list(unique.values())
print(f"Total sequences collected: {len(all_sequences)}")
print(f"Number of unique sequences: {len(unique_indices)}")

# Print heads of each
print("\nFirst 5 unique sequences:")
for idx in unique_indices[:5]:
    print(f"Sequence: {all_sequences[idx]}")
    print(f"Residual: {all_residuals[idx]}")
    print(f"Log odds: {all_passive_logodds[idx]}")
    print()

Total sequences collected: 3000
Number of unique sequences: 597

First 5 unique sequences:
Sequence: ('R', 'r', 'r', 'L', 'r', 'r')
Residual: tensor([[-6.0643e-01, -2.9450e-01, -2.8806e-01,  3.4194e-01, -2.4693e-01,
         -2.1717e-01],
        [ 8.3598e-01,  2.4567e-01,  5.2077e-02, -3.6750e-01, -6.2135e-02,
          7.6535e-04],
        [ 9.5053e-01,  6.6044e-01,  4.2233e-01, -3.8450e-01,  1.3962e-01,
          1.2356e-01],
        [-1.1273e+00, -2.7368e-01, -3.4774e-02,  2.4035e-01,  2.5457e-01,
          2.6306e-01]])
Log odds: 0.3434194814002114

Sequence: ('r', 'r', 'L', 'r', 'r', 'L')
Residual: tensor([[-0.1352, -0.1269,  0.4857, -0.1494, -0.2174,  0.4484],
        [ 0.0888, -0.0506, -0.5266, -0.1607, -0.1178, -0.4462],
        [ 0.4544,  0.4586, -0.4309,  0.1623,  0.1031, -0.6151],
        [-0.1666,  0.0886,  0.3513,  0.2872,  0.3233,  0.4983]])
Log odds: -3.0740335652556006

Sequence: ('r', 'L', 'r', 'r', 'L', 'l')
Residual: tensor([[-0.1352,  0.5919, -0.0977, -0.1595,  0.4

In [187]:
# 6. Find sequences with max and min log odds
logodds_array = np.array(all_passive_logodds)
max_idx = np.argmax(logodds_array)
min_idx = np.argmin(logodds_array)

print(f"Max log odds: {logodds_array[max_idx]:.4f}")
print(f"Min log odds: {logodds_array[min_idx]:.4f}")
print(f"Sequence for max log odds: {all_sequences[max_idx]}")
print(f"Sequence for min log odds: {all_sequences[min_idx]}")


Max log odds: 4.8012
Min log odds: -4.7861
Sequence for max log odds: ('R', 'R', 'R', 'R', 'R', 'R')
Sequence for min log odds: ('L', 'L', 'L', 'L', 'L', 'L')


In [188]:
# 7. Train linear regression: residual last column -> passive log odds
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# 1. Get unique indices, shuffle, and split
unique_indices = list(unique.values())
np.random.seed(42)
np.random.shuffle(unique_indices)
n_test = len(unique_indices) // 4
test_indices = unique_indices[:n_test]
train_indices = unique_indices[n_test:]

# 2. Prepare X (features) and y (targets)
def get_features_targets(indices):
    X = []
    y = []
    for idx in indices:
        # Residual: shape [n_embd, seq_len], take last column
        X.append(all_residuals[idx][:, -1].cpu().numpy())
        # Passive log odds: scalar value
        y.append(all_passive_logodds[idx])
    X = np.stack(X)
    y = np.array(y)
    return X, y

X_train, y_train = get_features_targets(train_indices)
X_test, y_test = get_features_targets(test_indices)

# 3. Train linear regression
reg = LinearRegression()
reg.fit(X_train, y_train)

# 4. Predict and evaluate
y_pred = reg.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"Test R^2: {r2:.4f}")

# 5. Print a few predictions vs. true values with sequences
print(f"\nFirst 5 test predictions:")
for i in range(5):
    idx = test_indices[i]
    print(f"Sequence: {all_sequences[idx]}")
    print(f"Predicted: {y_pred[i]:.4f}, True: {y_test[i]:.4f}")
    print()


Test R^2: 0.9782

First 5 test predictions:
Sequence: ('r', 'r', 'r', 'r', 'r', 'L')
Predicted: -3.0178, True: -2.8225

Sequence: ('r', 'r', 'R', 'r', 'R', 'R')
Predicted: 3.9186, True: 4.1078

Sequence: ('l', 'L', 'l', 'L', 'l', 'l')
Predicted: -1.4847, True: -1.4037

Sequence: ('l', 'l', 'r', 'l', 'l', 'l')
Predicted: -0.5777, True: -0.7725

Sequence: ('L', 'r', 'R', 'R', 'r', 'R')
Predicted: 3.2151, True: 3.5332

